In [24]:
>>> def car(env):
...    while True:
...        print('Start parking at %d' % env.now)
...        parking_duration = 5
...        yield env.timeout(parking_duration)
...
...        print('Start driving at %d' % env.now)
...        trip_duration = 2
...        yield env.timeout(trip_duration)

>>> import simpy
>>> env = simpy.Environment()
>>> env.process(car(env))
>>> env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [3]:
>>> class Car(object):
...     def __init__(self, env):
...         self.env = env
...         # Start the run process everytime an instance is created.
...         self.action = env.process(self.run())
...
...     def run(self):
...         while True:
...             print('Start parking and charging at %d' % self.env.now)
...             charge_duration = 5
...             # We yield the process that process() returns
...             # to wait for it to finish
...             yield self.env.process(self.charge(charge_duration))
...
...             # The charge process has finished and
...             # we can start driving again.
...             print('Start driving at %d' % self.env.now)
...             trip_duration = 2
...             yield self.env.timeout(trip_duration)
...
...     def charge(self, duration):
...         yield self.env.timeout(duration)
>>> import simpy
>>> env = simpy.Environment()
>>> car = Car(env)
>>> env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [8]:
>>> def driver(env, car):
...     yield env.timeout(3)
...     car.action.interrupt()
>>> class Car(object):
...     def __init__(self, env):
...         self.env = env
...         self.action = env.process(self.run())
...
...     def run(self):
...         while True:
...             print('Start parking and charging at %d' % self.env.now)
...             charge_duration = 5
...             # We may get interrupted while charging the battery
...             try:
...                 yield self.env.process(self.charge(charge_duration))
...             except simpy.Interrupt:
...                 # When we received an interrupt, we stop charging and
...                 # switch to the "driving" state
...                 print('Was interrupted. Hope, the battery is full enough ...')
...
...             print('Start driving at %d' % self.env.now)
...             trip_duration = 2
...             yield self.env.timeout(trip_duration)
...
...     def charge(self, duration):
...         yield self.env.timeout(duration)
>>> env = simpy.Environment()
>>> car = Car(env)
>>> env.process(driver(env, car))
>>> env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


In [9]:
>>> def car(env, name, bcs, driving_time, charge_duration):
...     # Simulate driving to the BCS
...     yield env.timeout(driving_time)
...
...     # Request one of its charging spots
...     print('%s arriving at %d' % (name, env.now))
...     with bcs.request() as req:
...         yield req
...
...         # Charge the battery
...         print('%s starting to charge at %s' % (name, env.now))
...         yield env.timeout(charge_duration)
...         print('%s leaving the bcs at %s' % (name, env.now))
>>> import simpy
>>> env = simpy.Environment()
>>> bcs = simpy.Resource(env, capacity=2)
>>> for i in range(4):
...     env.process(car(env, 'Car %d' % i, bcs, i*2, 5))
>>> env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [11]:
>>> import simpy
>>>
>>> def example(env):
...     value = yield env.timeout(1, value=42)
...     print('now=%d, value=%d' % (env.now, value))
>>>
>>> env = simpy.Environment()
>>> p = env.process(example(env))
>>> env.run()

now=1, value=42


In [13]:
>>> import simpy
>>>
>>> def example(env):
...     event = simpy.events.Timeout(env, delay=1, value=42)
...     value = yield event
...     print('now=%d, value=%d' % (env.now, value))
>>>
>>> env = simpy.Environment()
>>> example_gen = example(env)
>>> p = simpy.events.Process(env, example_gen)
>>>
>>> env.run()

now=1, value=42


In [15]:
>>> import simpy
>>>
>>> def my_proc(env):
...     yield env.timeout(1)
...     return 'Monty Python’s Flying Circus'
>>>
>>> env = simpy.Environment()
>>> proc = env.process(my_proc(env))
>>> env.run(until=proc)

'Monty Python’s Flying Circus'

In [17]:
>>> import simpy
>>>
>>> def my_callback(event):
...     print('Called back from', event)
...
>>> env = simpy.Environment()
>>> event = env.event()
>>> event.callbacks.append(my_callback)
>>> event.callbacks

[<function __main__.my_callback(event)>]

In [18]:
>>> class School:
...     def __init__(self, env):
...         self.env = env
...         self.class_ends = env.event()
...         self.pupil_procs = [env.process(self.pupil()) for i in range(3)]
...         self.bell_proc = env.process(self.bell())
...
...     def bell(self):
...         for i in range(2):
...             yield self.env.timeout(45)
...             self.class_ends.succeed()
...             self.class_ends = self.env.event()
...             print()
...
...     def pupil(self):
...         for i in range(2):
...             print(r' \o/', end='')
...             yield self.class_ends
...
>>> school = School(env)
>>> env.run()

 \o/ \o/ \o/
 \o/ \o/ \o/


In [21]:
>>> from simpy.util import start_delayed
>>>
>>> def sub(env):
...     yield env.timeout(1)
...     return 23
...
>>> def parent(env):
...     sub_proc = yield start_delayed(env, sub(env), delay=3)
...     ret = yield sub_proc
...     return ret
...
>>> env.run(env.process(parent(env)))

23

In [22]:
>>> from random import seed, randint
>>> seed(23)
>>>
>>> import simpy
>>>
>>> class EV:
...     def __init__(self, env):
...         self.env = env
...         self.drive_proc = env.process(self.drive(env))
...         self.bat_ctrl_proc = env.process(self.bat_ctrl(env))
...         self.bat_ctrl_reactivate = env.event()
...
...     def drive(self, env):
...         while True:
...             # Drive for 20-40 min
...             yield env.timeout(randint(20, 40))
...
...             # Park for 1–6 hours
...             print('Start parking at', env.now)
...             self.bat_ctrl_reactivate.succeed()  # "reactivate"
...             self.bat_ctrl_reactivate = env.event()
...             yield env.timeout(randint(60, 360))
...             print('Stop parking at', env.now)
...
...     def bat_ctrl(self, env):
...         while True:
...             print('Bat. ctrl. passivating at', env.now)
...             yield self.bat_ctrl_reactivate  # "passivate"
...             print('Bat. ctrl. reactivated at', env.now)
...
...             # Intelligent charging behavior here …
...             yield env.timeout(randint(30, 90))
...
>>> env = simpy.Environment()
>>> ev = EV(env)
>>> env.run(until=150)

Bat. ctrl. passivating at 0
Start parking at 29
Bat. ctrl. reactivated at 29
Bat. ctrl. passivating at 60
Stop parking at 131


In [23]:
>>> class EV:
...     def __init__(self, env):
...         self.env = env
...         self.drive_proc = env.process(self.drive(env))
...
...     def drive(self, env):
...         while True:
...             # Drive for 20-40 min
...             yield env.timeout(randint(20, 40))
...
...             # Park for 1–6 hours
...             print('Start parking at', env.now)
...             charging = env.process(self.bat_ctrl(env))
...             parking = env.timeout(randint(60, 360))
...             yield charging & parking
...             print('Stop parking at', env.now)
...
...     def bat_ctrl(self, env):
...         print('Bat. ctrl. started at', env.now)
...         # Intelligent charging behavior here …
...         yield env.timeout(randint(30, 90))
...         print('Bat. ctrl. done at', env.now)
...
>>> env = simpy.Environment()
>>> ev = EV(env)
>>> env.run(until=310)

Start parking at 29
Bat. ctrl. started at 29
Bat. ctrl. done at 83
Stop parking at 305


In [26]:
>>> import simpy
>>>
>>> def clock(env, name, tick):
...     while True:
...         print(name, env.now)
...         yield env.timeout(tick)
...
>>> env = simpy.Environment()
>>> env.process(clock(env, 'fast', 0.5))

>>> env.process(clock(env, 'slow', 1))

>>> env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


In [6]:
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


In [11]:
>>> def subfunc(env):
...     print(env.active_process)  # will print "p1"
>>>
>>> def my_proc(env):
...     while True:
...         print(env.active_process)  # will print "p1"
...         subfunc(env)
...         yield env.timeout(1)
>>>
>>> env = simpy.Environment()
>>> p1 = env.process(my_proc(env))
>>> env.active_process  # None
>>> env.step()

>>> env.active_process  # None

<Process(my_proc) object at 0x1e0c5f84790>
<Process(my_proc) object at 0x1e0c5f84790>


In [18]:
>>> import simpy
>>>
>>> def my_callback(event):
...     print('Called back from', event)
...
>>> env = simpy.Environment()
>>> event = env.event()
>>> event.callbacks.append(my_callback)
>>> event.callbacks

[<function __main__.my_callback(event)>]

In [21]:
>>> import simpy
>>>
>>> def resource_user(env, resource):
...     request = resource.request()  # Generate a request event
...     yield request                 # Wait for access
...     yield env.timeout(1)          # Do something
...     resource.release(request)     # Release the resource
...
>>> env = simpy.Environment()
>>> res = simpy.Resource(env, capacity=1)
>>> user = env.process(resource_user(env, res))

>>> def resource_user(env, resource):
...     with resource.request() as req:  # Generate a request event
...         yield req                    # Wait for access
...         yield env.timeout(1)         # Do something
...                                      # Resource released automatically
>>> user = env.process(resource_user(env, res))

>>> res = simpy.Resource(env, capacity=1)
>>>
>>> def print_stats(res):
...     print(f'{res.count} of {res.capacity} slots are allocated.')
...     print(f'  Users: {res.users}')
...     print(f'  Queued events: {res.queue}')
>>>
>>>
>>> def user(res):
...     print_stats(res)
...     with res.request() as req:
...         yield req
...         print_stats(res)
...     print_stats(res)
>>>
>>> procs = [env.process(user(res)), env.process(user(res))]
>>> env.run()

0 of 1 slots are allocated.
  Users: []
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x1e0c63bcb80>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x1e0c63bcb80>]
  Queued events: [<Request() object at 0x1e0c63bc250>]
0 of 1 slots are allocated.
  Users: []
  Queued events: [<Request() object at 0x1e0c63bc250>]
1 of 1 slots are allocated.
  Users: [<Request() object at 0x1e0c63bc250>]
  Queued events: []
0 of 1 slots are allocated.
  Users: []
  Queued events: []


In [22]:
>>> def resource_user(name, env, resource, wait, prio):
...     yield env.timeout(wait)
...     with resource.request(priority=prio) as req:
...         print(f'{name} requesting at {env.now} with priority={prio}')
...         yield req
...         print(f'{name} got resource at {env.now}')
...         yield env.timeout(3)
...
>>> env = simpy.Environment()
>>> res = simpy.PriorityResource(env, capacity=1)
>>> p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
>>> p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
>>> p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
>>> env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
3 got resource at 3
2 got resource at 6


In [23]:
>>> def resource_user(name, env, resource, wait, prio):
...     yield env.timeout(wait)
...     with resource.request(priority=prio) as req:
...         print(f'{name} requesting at {env.now} with priority={prio}')
...         yield req
...         print(f'{name} got resource at {env.now}')
...         try:
...             yield env.timeout(3)
...         except simpy.Interrupt as interrupt:
...             by = interrupt.cause.by
...             usage = env.now - interrupt.cause.usage_since
...             print(f'{name} got preempted by {by} at {env.now}'
...                   f' after {usage}')
...
>>> env = simpy.Environment()
>>> res = simpy.PreemptiveResource(env, capacity=1)
>>> p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
>>> p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
>>> p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
>>> env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
1 got preempted by <Process(resource_user) object at 0x1e0c63130d0> at 2 after 2
3 got resource at 2
2 got resource at 5


In [25]:
>>> def user(name, env, res, prio, preempt):
...     with res.request(priority=prio, preempt=preempt) as req:
...         try:
...             print(f'{name} requesting at {env.now}')
...             assert isinstance(env.now, int), type(env.now)
...             yield req
...             assert isinstance(env.now, int), type(env.now)
...             print(f'{name} got resource at {env.now}')
...             yield env.timeout(3)
...         except simpy.Interrupt:
...             print(f'{name} got preempted at {env.now}')
>>>
>>> env = simpy.Environment()
>>> res = simpy.PreemptiveResource(env, capacity=1)
>>> A = env.process(user('A', env, res, prio=0, preempt=True))
>>> env.run(until=1)  # Give A a head start
>>> B = env.process(user('B', env, res, prio=-2, preempt=False))
>>> C = env.process(user('C', env, res, prio=-1, preempt=True))
>>> env.run()

A requesting at 0
A got resource at 0
B requesting at 1
C requesting at 1
B got resource at 3
C got resource at 6


In [26]:
>>> class GasStation:
...     def __init__(self, env):
...         self.fuel_dispensers = simpy.Resource(env, capacity=2)
...         self.gas_tank = simpy.Container(env, init=100, capacity=1000)
...         self.mon_proc = env.process(self.monitor_tank(env))
...
...     def monitor_tank(self, env):
...         while True:
...             if self.gas_tank.level < 100:
...                 print(f'Calling tanker at {env.now}')
...                 env.process(tanker(env, self))
...             yield env.timeout(15)
>>>
>>>
>>> def tanker(env, gas_station):
...     yield env.timeout(10)  # Need 10 Minutes to arrive
...     print(f'Tanker arriving at {env.now}')
...     amount = gas_station.gas_tank.capacity - gas_station.gas_tank.level
...     yield gas_station.gas_tank.put(amount)
>>>
>>>
>>> def car(name, env, gas_station):
...     print(f'Car {name} arriving at {env.now}')
...     with gas_station.fuel_dispensers.request() as req:
...         yield req
...         print(f'Car {name} starts refueling at {env.now}')
...         yield gas_station.gas_tank.get(40)
...         yield env.timeout(5)
...         print(f'Car {name} done refueling at {env.now}')
>>>
>>>
>>> def car_generator(env, gas_station):
...     for i in range(4):
...         env.process(car(i, env, gas_station))
...         yield env.timeout(5)
>>>
>>>
>>> env = simpy.Environment()
>>> gas_station = GasStation(env)
>>> car_gen = env.process(car_generator(env, gas_station))
>>> env.run(35)

Car 0 arriving at 0
Car 0 starts refueling at 0
Car 1 arriving at 5
Car 0 done refueling at 5
Car 1 starts refueling at 5
Car 2 arriving at 10
Car 1 done refueling at 10
Car 2 starts refueling at 10
Calling tanker at 15
Car 3 arriving at 15
Car 3 starts refueling at 15
Tanker arriving at 25
Car 2 done refueling at 30
Car 3 done refueling at 30


In [27]:
>>> def producer(env, store):
...     for i in range(100):
...         yield env.timeout(2)
...         yield store.put(f'spam {i}')
...         print(f'Produced spam at', env.now)
>>>
>>>
>>> def consumer(name, env, store):
...     while True:
...         yield env.timeout(1)
...         print(name, 'requesting spam at', env.now)
...         item = yield store.get()
...         print(name, 'got', item, 'at', env.now)
>>>
>>>
>>> env = simpy.Environment()
>>> store = simpy.Store(env, capacity=2)
>>>
>>> prod = env.process(producer(env, store))
>>> consumers = [env.process(consumer(i, env, store)) for i in range(2)]
>>>
>>> env.run(until=5)

0 requesting spam at 1
1 requesting spam at 1
Produced spam at 2
0 got spam 0 at 2
0 requesting spam at 3
Produced spam at 4
1 got spam 1 at 4


In [28]:
>>> from collections import namedtuple
>>>
>>> Machine = namedtuple('Machine', 'size, duration')
>>> m1 = Machine(1, 2)  # Small and slow
>>> m2 = Machine(2, 1)  # Big and fast
>>>
>>> env = simpy.Environment()
>>> machine_shop = simpy.FilterStore(env, capacity=2)
>>> machine_shop.items = [m1, m2]  # Pre-populate the machine shop
>>>
>>> def user(name, env, ms, size):
...     machine = yield ms.get(lambda machine: machine.size == size)
...     print(name, 'got', machine, 'at', env.now)
...     yield env.timeout(machine.duration)
...     yield ms.put(machine)
...     print(name, 'released', machine, 'at', env.now)
>>>
>>>
>>> users = [env.process(user(i, env, machine_shop, (i % 2) + 1))
...          for i in range(3)]
>>> env.run()

0 got Machine(size=1, duration=2) at 0
1 got Machine(size=2, duration=1) at 0
1 released Machine(size=2, duration=1) at 1
0 released Machine(size=1, duration=2) at 2
2 got Machine(size=1, duration=2) at 2
2 released Machine(size=1, duration=2) at 4


In [29]:
>>> env = simpy.Environment()
>>> issues = simpy.PriorityStore(env)
>>>
>>> def inspector(env, issues):
...     for issue in [simpy.PriorityItem('P2', '#0000'),
...                   simpy.PriorityItem('P0', '#0001'),
...                   simpy.PriorityItem('P3', '#0002'),
...                   simpy.PriorityItem('P1', '#0003')]:
...         yield env.timeout(1)
...         print(env.now, 'log', issue)
...         yield issues.put(issue)
>>>
>>> def maintainer(env, issues):
...     while True:
...         yield env.timeout(3)
...         issue = yield issues.get()
...         print(env.now, 'repair', issue)
>>>
>>> _ = env.process(inspector(env, issues))
>>> _ = env.process(maintainer(env, issues))
>>> env.run()

1 log PriorityItem(priority='P2', item='#0000')
2 log PriorityItem(priority='P0', item='#0001')
3 log PriorityItem(priority='P3', item='#0002')
3 repair PriorityItem(priority='P0', item='#0001')
4 log PriorityItem(priority='P1', item='#0003')
6 repair PriorityItem(priority='P1', item='#0003')
9 repair PriorityItem(priority='P2', item='#0000')
12 repair PriorityItem(priority='P3', item='#0002')


In [32]:
>>> import time
>>> import simpy.rt
>>>
>>> def slow_proc(env):
...     time.sleep(0.02)  # Heavy computation :-)
...     yield env.timeout(1)
>>>
>>> env = simpy.rt.RealtimeEnvironment(factor=0.01)
>>> proc = env.process(slow_proc(env))
>>> try:
...     env.run(until=proc)
...     print('Everything alright')
... except RuntimeError:
...     print('Simulation is too slow')

Simulation is too slow


In [33]:
>>> import time
>>> import simpy
>>>
>>> def example(env):
...     start = time.perf_counter()
...     yield env.timeout(1)
...     end = time.perf_counter()
...     print('Duration of one simulation time unit: %.2fs' % (end - start))
>>>
>>> env = simpy.Environment()
>>> proc = env.process(example(env))
>>> env.run(until=proc)

Duration of one simulation time unit: 0.00s


In [34]:
>>> import simpy.rt
>>> env = simpy.rt.RealtimeEnvironment(factor=0.1)
>>> proc = env.process(example(env))
>>> env.run(until=proc)

Duration of one simulation time unit: 0.12s


In [35]:
>>> env = simpy.rt.RealtimeEnvironment(factor=0.01, strict=False)
>>> proc = env.process(slow_proc(env))
>>> try:
...     env.run(until=proc)
...     print('Everything alright')
... except RuntimeError:
...     print('Simulation is too slow')

Everything alright


In [36]:
>>> import simpy
>>>
>>> data = []  # This list will hold all collected data
>>>
>>> def test_process(env, data):
...     val = 0
...     for i in range(5):
...         val += env.now
...         data.append(val)  # Collect data
...         yield env.timeout(1)
>>>
>>> env = simpy.Environment()
>>> p = env.process(test_process(env, data))
>>> env.run(p)
>>> print('Collected', data)  # Lets see what we got

Collected [0, 1, 3, 6, 10]
